In [5]:
import csv  # Import the csv module

# Create some data
data = [
    "Hi. So I'm having trouble accessing my AT&T account online whenever I try to log in. It says my user name or password is incorrect. Can you help me to reset my password?",
    "Uh So I'm moving to a different state and won't be needing your service anymore. I need help to close my account.",
    "I just ordered a device from AT&T but I haven't received any updates on the delivery status. Can you give me an update real quick?",
    "Hi. Um I want to upgrade my phone to the newest iphone. 14 was 256 gigabytes. Uh, the version gold. Uh is it possible for me to purchase it through the installment plan?",
    "I'm a bit concerned about cyber attacks and the security of my personal information. I'm wondering what measures AT&T are taking right now to protect customers.",
    "Hello. I'm interested in signing up for the Loyal program. Can you tell me more about it? Like, what kind of benefit and discounts are, are available through this program?",
    "Ok. So my phone is cracked and I need to get a repair like right now, um, can you schedule an appointment today for me at the nearest repair center?",
    "Hi. I have an issue with my internet connection. It's been down for the past hour and, like, I really can't believe the service you're providing me is so useless.",
    "I've been having trouble with my internet service. Um, it's been going on for two days now. Uh, is it an outage in my area? I've tried resetting my modem and router several times already.",
    "Hello. I have some questions about my bill. I noticed that my bill is higher than usual this month. Can you explain why? I'm pretty sure I didn't use that much of data this month.",
    "Hello. Um, I'd like to upgrade my plan to include more data. I'm currently on a basic plan. What are the available plans right now?",
    "Hello. I'd like to add a new line to my account for my sister. Can you help me with that?",
    "Hello. I'm traveling to Europe next week and I'd like to use my phone there. What are my options for international roaming?",
    "Hi, I recently moved and I need to update my billing address. Thank you.",
    "Hello. I'm interested in switching to AT&T. So what are the promotions that are currently available?",]

# Create a new CSV file and write each line of text as a separate row
with open('transcribe_data.csv', 'w', newline='') as file:  # Open a new file for writing
    writer = csv.writer(file)    # Create a csv writer object
    writer.writerow(['Text'])  # add label to the row
    for line in data:  # Loop over each line of data
        writer.writerow([line])  # Write the line to a new row in the csv file

In [7]:
import boto3

# Define the text data to classify and the name of the CSV file
text_data = 'sentiment full.csv'

# Initialize the Amazon Comprehend client
comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')

# Define the list of department categories to classify the text into
categories = [
    'Billing and Payment',
    'Internet & Direct TV',
    'Sales & Account management',
    'Troubleshoot & repair',
    'Loyalty department'
]

# Open the CSV file and read the sentences
with open(text_data, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) # skip header row
    for row in reader:
        # Get the sentence from the row
        sentence = row[1]
        label = row[2]
        
        # Call the Comprehend API to detect sentiment
        response = comprehend.detect_sentiment(Text=sentence, LanguageCode='en')
        
        # Get the predicted sentiment
        predicted_sentiment = response['Sentiment']

        # Print the sentence, actual label, predicted label, and confidence scores
        print(f"Sentence: {sentence}")
        print(f"Actual label: {label}")
        print(f"Predicted label: {predicted_sentiment}")
        print(f"Positive score: {response['SentimentScore']['Positive']:.2f}")
        print(f"Negative score: {response['SentimentScore']['Negative']:.2f}")
        print(f"Neutral score: {response['SentimentScore']['Neutral']:.2f}")
        print(f"Mixed score: {response['SentimentScore']['Mixed']:.2f}")
        print()

Sentence: Hi, I'm having trouble accessing my AT&T account online. Whenever I try to log in, it says my username or password is incorrect. Can you help me to reset my password.
Actual label: neutral
Predicted label: NEGATIVE
Positive score: 0.01
Negative score: 0.67
Neutral score: 0.32
Mixed score: 0.01

Sentence: Hi, I have an issue with my internet connection. It's been down for the past hour. I can't believe the service you're providing me is so useless!
Actual label: super negative
Predicted label: NEGATIVE
Positive score: 0.00
Negative score: 1.00
Neutral score: 0.00
Mixed score: 0.00

Sentence: I've been having trouble with my internet service. It's been going on for two days now. Is there an outage in my area? I've tried resetting my modem and router several times already.
Actual label: neutral
Predicted label: NEGATIVE
Positive score: 0.07
Negative score: 0.90
Neutral score: 0.03
Mixed score: 0.01

Sentence: Hello, I have some questions about my bill. I noticed that my bill is 

In [10]:
import pandas as pd

# Open the CSV file and read the sentences
df = pd.read_csv(text_data, names=['input', 'sentiment'], header=0)

# Initialize empty lists to store predicted labels and scores
predicted_labels = []
predicted_scores = []
calculated_labels = []

# Open the CSV file and read the sentences
with open(text_data, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) # skip header row
    for row in reader:
        # Get the sentence from the row
        sentence = row[1]
        label = row[2]
        
        # Call the Comprehend API to detect sentiment
        response = comprehend.detect_sentiment(Text=sentence, LanguageCode='en')
    
        # Append the predicted label and score to the corresponding lists
        predicted_label = response['Sentiment']
        predicted_score = response['SentimentScore'][predicted_label.title()]
        predicted_labels.append(predicted_label)
        predicted_scores.append(predicted_score)
        
        # Calculate the new label based on the predicted score
        if predicted_label in ['NEGATIVE']: 
            if predicted_score >= 0.8: # we consider 0.8 as the bound
                calculated_label = 'super negative'
            else:
                calculated_label = 'negative'
        else:
            calculated_label = 'neutral'
        
        calculated_labels.append(calculated_label)

# Add the predicted labels and scores to the dataframe
df['Predicted Label'] = predicted_labels
df['Score'] = predicted_scores
df['Calculated Label'] = calculated_labels

# Print the final dataframe
print(df)


                                                input       sentiment  \
1   Hi, I'm having trouble accessing my AT&T accou...         neutral   
2   Hi, I have an issue with my internet connectio...  super negative   
3   I've been having trouble with my internet serv...         neutral   
4   Hello, I have some questions about my bill. I ...        negative   
5   Hi, I'd like to upgrade my plan to include mor...         neutral   
6   Hi, I'd like to add a new line to my account f...         neutral   
7   Hello, I'm traveling to Europe next week and I...         neutral   
8   Hi, I recently moved and I need to update my b...         neutral   
9   Hello, I'm interested in switching to ATT. Wha...         neutral   
10  I am moving to a different state and won't be ...         neutral   
11  I just ordered a device from ATT but I haven't...         neutral   
12  I want to upgrade my phone to the newest iPhon...         neutral   
13  I’m a bit concerned about cyberattacks and the.